# Eight Queens Puzzle: An ILP Approach
## Mohamad Fares El Hajj Chehade

In [1]:
import pandas as pd
import pulp as pl

In [2]:
n = 8 # number of queens 
rows = range(n)
columns = range(n)

In [3]:
# create the optimization problem with the decision variables 

m = pl.LpProblem('Chess_Queens', pl.LpMinimize)

x = pl.LpVariable.dicts(name="Position", indices=(rows,columns),cat='Binary')

In [4]:
# dummy objective function 

m += ( 5, 'Dummy Objective Function') ;

In [5]:
# n queens on the board

m += pl.lpSum(x[i][j] for i in rows for j in columns) == n

In [6]:
# at most one queen on each row and each column

for i in rows:
    m += pl.lpSum(x[i][j] for j in columns) <= 1

for j in columns:
    m += pl.lpSum(x[i][j] for i in rows) <= 1

In [7]:
# at most one queen on each diagonal 

for i in rows:
    for j in columns:
        lower_bound = max(-i, -j)
        upper_bound = min((n-1)-i, (n-1)-j)
        sum_range = range(lower_bound,upper_bound+1)
        m += pl.lpSum(x[i+k][j+k] for k in sum_range) <= 1
        

for i in rows:
    for j in columns: 
        lower_bound = max(i-(n-1), -j)
        upper_bound = min(i, (n-1)-j)
        sum_range = range(lower_bound,upper_bound+1)
        m += pl.lpSum(x[i-k][j+k] for k in sum_range) <= 1

In [8]:
flag = True

count = 0 # counts the solution number

L_previous = [] # stores the 8 positions of the previous solution, each position (row,column) in a tuple
D_solutions = {} # stores the solution number (key) and the corresponding array solution (value)

In [9]:
# map a feasible solution to True and an infeasible solution to false 

def check(status):
    if status == 1:
        return True
    else:
        return False 

In [10]:
while(flag): # as long as the solution is feasible, check the next one
    count += 1 # indicates the solution number
     
    if count != 1: # if this is not the first solution, make sure that the previous solution is no longer possible 
        m += pl.lpSum(x[index[0]][index[1]] for index in L_previous) <= n-1
        
        
    status = m.solve() # solve the problem
    flag = check(status) # check if there remains a feasible solution 
    
    
    if flag: # if there is a feasible solution
        L_previous = [] # re-initialize the array to store the new positions
        arr = [[0 for i in range(len(columns))] for j in range(len(rows))] # initialize the 2d array to store the entire sol
        
        for v in m.variables():
            
            indices = v.name.split("_")
            if indices[0] == "Position":
                
                value = v.varValue 
                row = int(indices[1]) 
                column = int(indices[2])
                
                arr[row][column] = int(value) # update the 2d array solution
            
                if value != 0: # checks if the value is 1
                    L_previous.append((row,column)) # add the 8 new non-zero positions

        D_solutions[count] = arr # store the solution in the dictionary of solutions

In [13]:
text = ''
for element in D_solutions:
    text += "Solution "+str(element)+":"+"\n\n"
    text += '\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in arr])+"\n"
    text += '\n\n'
print(text[:-2]) # -2 removes the extra spacing at the end 

Solution 1:

   0   0   0   0   1   0   0   0
   0   0   0   0   0   0   1   0
   0   1   0   0   0   0   0   0
   0   0   0   0   0   1   0   0
   0   0   1   0   0   0   0   0
   1   0   0   0   0   0   0   0
   0   0   0   1   0   0   0   0
   0   0   0   0   0   0   0   1


Solution 2:

   0   0   0   0   1   0   0   0
   0   0   0   0   0   0   1   0
   0   1   0   0   0   0   0   0
   0   0   0   0   0   1   0   0
   0   0   1   0   0   0   0   0
   1   0   0   0   0   0   0   0
   0   0   0   1   0   0   0   0
   0   0   0   0   0   0   0   1


Solution 3:

   0   0   0   0   1   0   0   0
   0   0   0   0   0   0   1   0
   0   1   0   0   0   0   0   0
   0   0   0   0   0   1   0   0
   0   0   1   0   0   0   0   0
   1   0   0   0   0   0   0   0
   0   0   0   1   0   0   0   0
   0   0   0   0   0   0   0   1


Solution 4:

   0   0   0   0   1   0   0   0
   0   0   0   0   0   0   1   0
   0   1   0   0   0   0   0   0
   0   0   0   0   0   1   0   0
   0   0   1   0  

In [19]:
# write the solution into a text file

#open text file
text_file = open(str(n)+" Queens Puzzle Solutions.txt", "w")
 
#write string to file
text_file.write(text)
 
#close file
text_file.close()